In [2]:
from dataclasses import dataclass
from pyrs.pyrs import passes, to_hugr_dot, to_hugr, check_soundness
from pytket.circuit import Circuit, OpType
from pathlib import Path
from pytket.qasm.qasm import circuit_from_qasm
from pytket.passes.auto_rebase import auto_rebase_pass
from pytket.passes import FlattenRelabelRegistersPass, RemoveRedundancies
from pytket.utils.stats import gate_counts
from pytket.utils.results import compare_unitaries
import graphviz


In [3]:
direc = Path.home() / "tket/tket/pytket/tests/qasm_test_files/"

files = list(direc.iterdir())

In [13]:

def view_dot(c: Circuit):
    # print(to_hugr_dot(c))
    display(graphviz.Source(to_hugr_dot(c)))


def compare(c: Circuit, c2: Circuit) -> bool:
    from pytket.extensions.qiskit import AerUnitaryBackend
    b = AerUnitaryBackend()
    (u, u2) = (b.run_circuit(circ).get_unitary() for circ in (c, c2))

    res =  compare_unitaries(u, u2)
    # if not res:
    #     print(u)
    #     print(u2)

    return res

In [20]:
def test_file(file: Path):
    try:
        c = circuit_from_qasm(file)
    except Exception as e:
        print(e)
        return
    try:
        auto_rebase_pass({OpType.PhasedX, OpType.ZZPhase, OpType.Rz}).apply(c)
    except RuntimeError as e:
        if "CustomGate" in str(e):
            return
        else:
            raise e

    # tket2 expects simple register names for now
    FlattenRelabelRegistersPass().apply(c)
    RemoveRedundancies().apply(c)
    print(f"{file.name}")
    c2, moves = passes.greedy_depth_reduce(c)
    assert gate_counts(c) == gate_counts(c2)
    print(f"{moves} {c.depth()} {c2.depth()}")

    if c.n_qubits < 9 and c.n_gates < 1000:
        if not compare(c, c2):
            print("unitary fail")
            for circ in (c, c2):
                print("\n".join(map(str,circ.get_commands())))
                print("\n")

In [21]:
# for f in files:
#     test_file(f)

test_file(direc/"test1.qasm")

test1.qasm
8 64 62
unitary fail
Rz(1) q[0];
Rz(1) q[1];
PhasedX(3, 1.75) q[2];
Rz(1.5) q[3];
PhasedX(0.5, 0.5) q[0];
PhasedX(0.5, 0.5) q[1];
PhasedX(0.0375, 0) q[3];
ZZPhase(0.0375) q[0], q[1];
PhasedX(2.5, 0) q[3];
Rz(0.5) q[0];
Rz(0.5) q[1];
PhasedX(0.5, 0.5) q[0];
PhasedX(0.5, 0.5) q[1];
PhasedX(0.5, 0) q[0];
PhasedX(0.5, 0) q[1];
ZZPhase(0.0375) q[0], q[1];
Rz(0.5) q[0];
Rz(1.5) q[1];
ZZPhase(0.5) q[0], q[3];
Rz(0.5) q[0];
Rz(0.5) q[3];
PhasedX(0.5, 0.5) q[0];
PhasedX(0.5, 0.5) q[3];
PhasedX(0.5, 0) q[0];
PhasedX(0.5, 0) q[3];
Rz(1) q[0];
Rz(2.5) q[3];
PhasedX(0.5, 0.5) q[0];
PhasedX(1.9625, 0) q[3];
PhasedX(1, 0.25) q[0];
Rz(1.5) q[3];
ZZPhase(0.5) q[0], q[1];
Rz(0.5) q[0];
Rz(0.5) q[1];
PhasedX(0.5, 0.5) q[0];
PhasedX(0.5, 0.5) q[1];
PhasedX(0.5, 0) q[0];
PhasedX(0.5, 0) q[1];
Rz(1) q[0];
PhasedX(3.5, 0.5) q[1];
PhasedX(3.5, 0.5) q[0];
Rz(1.5) q[1];
ZZPhase(3.5) q[1], q[2];
Rz(0.5) q[1];
Rz(0.5) q[2];
PhasedX(0.5, 0.5) q[1];
PhasedX(0.5, 0.5) q[2];
PhasedX(0.5, 0) q[1];
PhasedX(0